In [ ]:
# version of esm
%pip show fair-esm


In [1]:
#import esm
import torch
import numpy as np
import pandas as pd
import loralib as lora
import esmLoRA.esm as el
from esmLoRA.esm.model.esm2 import ESM2
import pytorch_lightning as pl
import math



In [2]:
# Load ESM model

#model used on this exemple available on git, most recent version 

pretrained_model, alphabet = el.pretrained.esm2_t6_8M_UR50D()
#model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()

batch_converter = alphabet.get_batch_converter()

# disables dropout for deterministic results
pretrained_model.eval()


ESM2(
  (embed_tokens): Embedding(33, 320, padding_idx=1)
  (layers): ModuleList(
    (0): TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=320, out_features=320, bias=True)
        (v_proj): Linear(in_features=320, out_features=320, bias=True)
        (q_proj): Linear(in_features=320, out_features=320, bias=True)
        (out_proj): Linear(in_features=320, out_features=320, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=320, out_features=1280, bias=True)
      (fc2): Linear(in_features=1280, out_features=320, bias=True)
      (final_layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
    )
    (1): TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=320, out_features=320, bias=True)
        (v_proj): Linear(in_features=320, out_features=320, bias=True)
        (

In [3]:
# PATH = 'esm2_t6_8M_UR50D_pretrained_checkpoint.pt'
# torch.save(pretrained_model.state_dict(), PATH)

In [3]:
data = [("protein1", "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG")]
batch_labels, batch_strs, batch_tokens = batch_converter(data)
batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
batch_tokens

tensor([[ 0, 20, 15, 11,  7, 10, 16,  9, 10,  4, 15,  8, 12,  7, 10, 12,  4,  9,
         10,  8, 15,  9, 14,  7,  8,  6,  5, 16,  4,  5,  9,  9,  4,  8,  7,  8,
         10, 16,  7, 12,  7, 16, 13, 12,  5, 19,  4, 10,  8,  4,  6, 19, 17, 12,
          7,  5, 11, 14, 10,  6, 19,  7,  4,  5,  6,  6,  2]])

In [4]:
batch_tokens[0]
pretrained_model(batch_tokens, return_contacts=False)


{'logits': tensor([[[ 13.9791,  -9.0924,  -6.5645,  ..., -14.8934, -15.2000,  -9.0806],
          [ -8.2908, -14.3483,  -9.2373,  ..., -15.6513, -15.9180, -14.3464],
          [-13.0977, -23.2342, -13.0121,  ..., -16.0929, -16.0443, -23.2163],
          ...,
          [-11.8081, -23.6640, -13.1323,  ..., -16.5725, -16.6506, -23.6673],
          [-11.6440, -21.9399, -11.6811,  ..., -16.2364, -16.2875, -21.9438],
          [ -5.6318,  -6.4352,  19.2604,  ..., -16.4212, -16.2883,  -6.4788]]],
        grad_fn=<AddBackward0>),
 'representations': {}}

In [5]:
model = ESM2(num_layers=6, embed_dim=320, attention_heads=20, token_dropout=False)

In [6]:
PATH = 'esm2_t6_8M_UR50D_pretrained_checkpoint.pt'

model.load_state_dict(torch.load(PATH), strict=False)


_IncompatibleKeys(missing_keys=['layers.0.fc1.lora_A', 'layers.0.fc1.lora_B', 'layers.0.fc2.lora_A', 'layers.0.fc2.lora_B', 'layers.1.fc1.lora_A', 'layers.1.fc1.lora_B', 'layers.1.fc2.lora_A', 'layers.1.fc2.lora_B', 'layers.2.fc1.lora_A', 'layers.2.fc1.lora_B', 'layers.2.fc2.lora_A', 'layers.2.fc2.lora_B', 'layers.3.fc1.lora_A', 'layers.3.fc1.lora_B', 'layers.3.fc2.lora_A', 'layers.3.fc2.lora_B', 'layers.4.fc1.lora_A', 'layers.4.fc1.lora_B', 'layers.4.fc2.lora_A', 'layers.4.fc2.lora_B', 'layers.5.fc1.lora_A', 'layers.5.fc1.lora_B', 'layers.5.fc2.lora_A', 'layers.5.fc2.lora_B'], unexpected_keys=[])

In [7]:
pretrained_model.load_state_dict(torch.load(PATH), strict=False)

<All keys matched successfully>

In [8]:
#Generate mapping for tokens to amino acids
token_mapping = alphabet.to_dict() # {amino_acid: token_id}
token_mapping = {v: k for k, v in token_mapping.items()} # {token_id: amino_acid}
token_mapping

{0: '<cls>',
 1: '<pad>',
 2: '<eos>',
 3: '<unk>',
 4: 'L',
 5: 'A',
 6: 'G',
 7: 'V',
 8: 'S',
 9: 'E',
 10: 'R',
 11: 'T',
 12: 'I',
 13: 'D',
 14: 'P',
 15: 'K',
 16: 'Q',
 17: 'N',
 18: 'F',
 19: 'Y',
 20: 'M',
 21: 'H',
 22: 'W',
 23: 'C',
 24: 'X',
 25: 'B',
 26: 'U',
 27: 'Z',
 28: 'O',
 29: '.',
 30: '-',
 31: '<null_1>',
 32: '<mask>'}

In [9]:
input_tokens = torch.Tensor([alphabet.encode("<cls>MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG<eos><pad><pad><pad><pad>")]).to(torch.int64)
input_tokens

tensor([[ 0, 20, 15, 11,  7, 10, 16,  9, 10,  4, 15,  8, 12,  7, 10, 12,  4,  9,
         10,  8, 15,  9, 14,  7,  8,  6,  5, 16,  4,  5,  9,  9,  4,  8,  7,  8,
         10, 16,  7, 12,  7, 16, 13, 12,  5, 19,  4, 10,  8,  4,  6, 19, 17, 12,
          7,  5, 11, 14, 10,  6, 19,  7,  4,  5,  6,  6,  2,  1,  1,  1,  1]])

In [10]:
input_tokens = torch.Tensor([alphabet.encode("<cls>M<mask>TVRQERLKSIVRILER<mask>SKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRG<mask>YVLA<mask>G<eos><pad><pad><pad><pad>")], )

input_tokens.to(torch.int64)

#pull logits from model
var = pretrained_model.forward(input_tokens.int(), return_contacts = False)['logits']
prediction = var[0,:].argmax(dim=1)

reconstructed = ''
for token in prediction:
    reconstructed += token_mapping[token.item()]
reconstructed


'<cls>MSTVRQERLKSIVRILERHSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGGYVLAEG<eos>EEEE'

In [11]:
var[0,:].argmax(dim=1)

tensor([ 0, 20,  8, 11,  7, 10, 16,  9, 10,  4, 15,  8, 12,  7, 10, 12,  4,  9,
        10, 21,  8, 15,  9, 14,  7,  8,  6,  5, 16,  4,  5,  9,  9,  4,  8,  7,
         8, 10, 16,  7, 12,  7, 16, 13, 12,  5, 19,  4, 10,  8,  4,  6, 19, 17,
        12,  7,  5, 11, 14, 10,  6,  6, 19,  7,  4,  5,  9,  6,  2,  9,  9,  9,
         9])

In [12]:
#pull logits from model
var = model.forward(input_tokens.int())['logits']
prediction = var[0,:].argmax(dim=1)

reconstructed = ''
for token in prediction:
#    print(token)
    reconstructed += token_mapping[token.item()]
reconstructed



SyntaxError: invalid syntax (3360133998.py, line 2)

In [13]:
lora.mark_only_lora_as_trainable(model)

In [14]:
from Bio import SeqIO
import pandas as pd
#read fasta file to csv with seqio
fasta_file = 'data/uniprotkb_reviewed_true_AND_annotation_2023_07_12.fasta'

with open(fasta_file) as fasta_file:  # Will close handle cleanly
    id = []
    sequence = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        id.append(seq_record.id)
        sequence.append(str(seq_record.seq))

tuples = list(zip(id, sequence))

#replace 15% of amino acids with mask token

masked_sequence = []


masked_tuples = list(zip(id, masked_sequence))

# remove length > 1024
tuples = [t for t in tuples if len(t[1]) <= 1022]

#append cls and eos tokens to each sequence (NOT NEEDED)
#tuples = [(t[0], "<cls>" + t[1] + "<eos>") for t in tuples]


# df = pd.DataFrame(tuples, columns=['id', 'sequence'])
# df = df[df['sequence'].str.len() <= 1022]
# df

,id,sequence
0,sp|A0A0B4J2F0|PIOS1_HUMAN,MFRRLTFAQLLFATVLGIAGGVYIFQPVFEQYAKDQKELKEKMQLV...
1,sp|A0A0C5B5G6|MOTSC_HUMAN,MRWQEMGYIFYPRKLR
2,sp|A0A0K2S4Q6|CD3CH_HUMAN,MTQRAGAAMLPSALLLLCVPGCLTVSGPSTVMGAVGESLSVQCRYE...
3,sp|A0A0U1RRE5|NBDY_HUMAN,MGDQPCASGRSTLPPGNAREAKPPKKRCLLAPRWDYPEGTPNGGST...
4,sp|A0A1B0GTW7|CIROP_HUMAN,MLLLLLLLLLLPPLVLRVAASRCLHDETQKSVSLLRPPFSQLPSKS...
...,...,...
12032,sp|Q9Y6X8|ZHX2_HUMAN,MASKRKSTTPCMVRTSQVVEQDVPEEVDRAKEKGIGTPQPDVAKDS...
12033,sp|Q9Y6Y0|NS1BP_HUMAN,MIPNGYLMFEDENFIESSVAKLNALRKSGQFCDVRLQVCGHEMLAH...
12034,sp|Q9Y6Y8|S23IP_HUMAN,MAERKPNGGSGGASTSSSGTNLLFSSSATEFSFNVPFIPVTQASAS...
12035,sp|Q9Y6Y9|LY96_HUMAN,MLPFLFFSTLFSSIFTEAQKQYWVCNSSDASISYTYCDKMQYPISI...


In [2]:
data = tuples
batch_labels, batch_strs, batch_tokens = batch_converter(data)
batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)
len(batch_tokens)

NameError: name 'tuples' is not defined

In [71]:
#Choose 0.15 of tokens to mask
for i in range(0, len(batch_tokens[0:3])):
    index_to_change = (np.random.choice(len(batch_tokens[i]), size=int(0.15*len(batch_tokens[i])), replace=False))
    


[ 974  536  738  833  116   71  739  710  280  608  847   51  385   25
  506  327  223  238  222  304  574  248  757  923  618  194   56  905
  662  831  529  488  172  136  183  512  471  496  398  699  215  515
  992  783  924  474  343  854  598  601  296  243  639 1022  589  914
  882  388  953  641  673  995  351  793  570  300  233  510  319  846
  184  855  267  467  100   49  330  829 1011  767  278   29  239  414
  554  256  700  192  719  723  594  202  617  160  159  632  271  776
  227  759  431  152  216  975  607  844  899  817  746  727  796  921
  910  242  753  380  264  788  721  112  720  148  983  778  542  786
  576  858  421  221  708  447  365   22   90  821  763  915  125  465
   63  665  893    9  954  903  354  866 1004  666  568  324  643]
[ 198  971  574  571  371  132  370  771  780  702  273  947  156  262
  514  785  324  580  705 1002   22  584  497  340  833  166  759  666
   79  809  922  546  788   67  641  627  277  456 1013  883  435  384
  648  683

In [17]:
attention_mask = torch.ones_like(batch_tokens)
#make sure attention mask is 0 for padding tokens "1"
attention_mask = attention_mask.masked_fill(batch_tokens == alphabet.padding_idx, 0)
attention_mask

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

In [36]:
batch_tokens[0:3]

tensor([[ 0, 20, 18,  ...,  1,  1,  1],
        [ 0, 20, 10,  ...,  1,  1,  1],
        [ 0, 20, 11,  ...,  1,  1,  1]])

In [40]:
model.forward(batch_tokens[0:3])['logits'].argmax(dim=2)


tensor([[ 0, 20, 18,  ...,  4, 12,  6],
        [ 0, 20, 10,  ...,  4,  4,  4],
        [ 0, 20, 11,  ...,  4,  4,  8]])

In [41]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)


ESM2(
  (embed_tokens): Embedding(33, 320, padding_idx=1)
  (layers): ModuleList(
    (0): TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=320, out_features=320, bias=True)
        (v_proj): Linear(in_features=320, out_features=320, bias=True)
        (q_proj): Linear(in_features=320, out_features=320, bias=True)
        (out_proj): Linear(in_features=320, out_features=320, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=320, out_features=1280, bias=True)
      (fc2): Linear(in_features=1280, out_features=320, bias=True)
      (final_layer_norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
    )
    (1): TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=320, out_features=320, bias=True)
        (v_proj): Linear(in_features=320, out_features=320, bias=True)
        (

In [43]:
from esmLoRA.esm.model.esm2 import ESM2
from torch.utils.data import random_split
from torch.utils.data import DataLoader

# data
dataset = batch_tokens
train, val = random_split(dataset, [math.ceil(0.9*len(batch_tokens)), math.floor(0.1*len(batch_tokens))])

train_loader = DataLoader(train, batch_size=32)
val_loader = DataLoader(val, batch_size=32)

# model
model = ESM2(num_layers=6, embed_dim=320, attention_heads=20, token_dropout=False)

# training
trainer = pl.Trainer(precision='bf16-mixed', limit_train_batches=0.5, max_epochs=10)
trainer.fit(model, train_loader, val_loader)
    


Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name                 | Type                  | Params
---------------------------------------------------------------
0 | embed_tokens         | Embedding             | 10.6 K
1 | layers               | ModuleList            | 7.7 M 
2 | contact_head         | ContactPredictionHead | 121   
3 | emb_layer_norm_after | LayerNorm             | 640   
4 | lm_head              | RobertaLMHead         | 113 K 
---------------------------------------------------------------
2.9 M     Trainable params
4.9 M     Non-trainable params
7.8 M     Total params
31.279    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/stor/home/alf3564/miniconda3/envs/esm/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 112 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


ValueError: too many values to unpack (expected 2)

In [ ]:
import torch
from torch import nn
import pytorch_lightning as pl
from torch.utils.data import DataLoader, random_split
from torch.nn import functional as F
from torchvision.datasets import MNIST
from torchvision import datasets, transforms
import os







class MNISTDataModule(pl.LightningDataModule):

  def setup(self, stage):
    # transforms for images
    transform=transforms.Compose([transforms.ToTensor(), 
                                  transforms.Normalize((0.1307,), (0.3081,))])
      
    # prepare transforms standard to MNIST
    self.mnist_train = MNIST(os.getcwd(), train=True, download=True, transform=transform)
    self.mnist_test = MNIST(os.getcwd(), train=False, download=True, transform=transform)

  def train_dataloader(self):
    return DataLoader(self.mnist_train, batch_size=64)

  def val_dataloader(self):
    return DataLoader(self.mnist_test, batch_size=64)

data_module = MNISTDataModule()



In [132]:
# train
model = pretrained_model()
trainer = pl.Trainer()

trainer.fit(model, data_module)

TypeError: forward() missing 1 required positional argument: 'tokens'